## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gKey

# Configure gmaps API key
gmaps.configure(api_key=gKey)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,75,0,4.61,clear sky
1,1,Hilo,US,19.7297,-155.0900,83.25,70,50,5.01,scattered clouds
2,2,New Norfolk,AU,-42.7826,147.0587,58.91,57,84,7.00,light rain
3,3,Namatanai,PG,-3.6667,152.4333,83.57,72,20,8.57,few clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,57.20,67,75,12.66,broken clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minT = float(input("Enter the minimum max temperature you would like to have in your vacation: "))
maxT = float(input("Enter the maximum max temperature you would like to have in your vacation: "))

Enter the minimum max temperature you would like to have in your vacation: 75
Enter the maximum max temperature you would like to have in your vacation: 85


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefferedCities = city_data_df.loc[(city_data_df["Max Temp"] >= minT) & (city_data_df["Max Temp"] <= maxT)]

In [19]:
# 4a. Determine if there are any empty rows.
prefferedCities.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefferedCities.dropna()

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,83.25,scattered clouds,19.7297,-155.0900,
3,Namatanai,PG,83.57,few clouds,-3.6667,152.4333,
7,Nalut,LY,83.84,clear sky,30.3333,10.8500,
11,Majene,ID,76.37,scattered clouds,-3.5403,118.9707,
14,Ormara,PK,78.69,overcast clouds,25.2088,64.6357,
15,Bathsheba,BB,77.61,broken clouds,13.2167,-59.5167,
16,Shenjiamen,CN,76.84,overcast clouds,29.9576,122.2980,
17,Fort Nelson,CA,76.59,scattered clouds,58.8053,-122.7002,
20,Amalapuram,IN,76.95,overcast clouds,16.5833,82.0167,
26,Hithadhoo,MV,81.66,overcast clouds,-0.6000,73.0833,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gKey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Hlat = row["Lat"]
    Hlng = row["Lng"]
    params["location"] = f"{Hlat},{Hlng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotelName = requests.get(baseURL, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotelName["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping.")
        

Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.


In [44]:
# 7. Drop the rows where there is no Hotel Name.
cleanHotels = hotel_df.dropna()
cleanHotels.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,83.25,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Namatanai,PG,83.57,few clouds,-3.6667,152.4333,Seagulls Inn Namatani
11,Majene,ID,76.37,scattered clouds,-3.5403,118.9707,Kost Putri RABBANI
14,Ormara,PK,78.69,overcast clouds,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
15,Bathsheba,BB,77.61,broken clouds,13.2167,-59.5167,Atlantis Hotel
16,Shenjiamen,CN,76.84,overcast clouds,29.9576,122.2980,Jinjiang Inn Zhoushan Putuo Shenjiamen
17,Fort Nelson,CA,76.59,scattered clouds,58.8053,-122.7002,The Fort Nelson Hotel
20,Amalapuram,IN,76.95,overcast clouds,16.5833,82.0167,swayamprabha service apartments
26,Hithadhoo,MV,81.66,overcast clouds,-0.6000,73.0833,Scoop Guest House
31,Guerrero Negro,MX,81.21,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"


In [46]:
# 8a. Create the output File (CSV)
outputFile = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
cleanHotels.to_csv(outputFile, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotelInfo = [info_box_template.format(**row) for index, row in cleanHotels.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleanHotels[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotelInfo)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))